In [1]:
import numpy as np
import pandas as pd

In [13]:
df=pd.read_csv(r"Foam Data.csv")

In [14]:
df=df.iloc[0:,0:4]

In [15]:
from sklearn.preprocessing import OneHotEncoder

In [16]:
categorical_cols = ['Techniques', 'Composition', 'Passes']
encoded_cols = pd.get_dummies(df[categorical_cols], drop_first=False)
df = pd.concat([df, encoded_cols], axis=1)


In [17]:
df=df.iloc[:,3:]

In [18]:
df

,R1,Passes,Techniques_Buired,Techniques_Groove,Techniques_Sandwich,Composition_A,Composition_B,Composition_C
0,1.43,1,True,False,False,True,False,False
1,0.90,2,True,False,False,True,False,False
2,0.32,4,True,False,False,True,False,False
3,1.76,1,True,False,False,True,False,False
4,0.99,2,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...
76,1.94,2,False,True,False,False,False,True
77,1.78,4,False,True,False,False,False,True
78,2.50,1,False,True,False,False,False,True
79,1.73,2,False,True,False,False,False,True


In [19]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, KFold

features = ['Passes', 'Techniques_Buired', 'Techniques_Groove', 'Techniques_Sandwich', 'Composition_A', 'Composition_B', 'Composition_C']
target = 'R1'
sensitivity_rank = []
r2_scores = []
parameters_taken = []
sensitivity_values = []
cv = KFold(n_splits=5, shuffle=True, random_state=42)
X_all = df[features]
y_all = df[target]
param_grid = {
    'alpha': [0.1, 1.0, 10.0], 
    'fit_intercept': [True, False], 
    'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
    'max_iter': [100, 200, 500], 
    'tol': [1e-3, 1e-4, 1e-5] 
}
grid_search = GridSearchCV(estimator=Ridge(), param_grid=param_grid, cv=cv, scoring='r2')
grid_search.fit(X_all, y_all)
r2_all_features = cross_val_score(grid_search.best_estimator_, X_all, y_all, cv=cv, scoring='r2').mean()

for feature in features:
    features_without_feature = [col for col in features if col != feature]
    X = df[features_without_feature]
    y = df[target]
    grid_search = GridSearchCV(estimator=Ridge(), param_grid=param_grid, cv=cv, scoring='r2')
    grid_search.fit(X, y)
    
    best_dtr = grid_search.best_estimator_
    r2_without_feature = cross_val_score(best_dtr, X, y, cv=cv, scoring='r2').mean() 
    
    sensitivity = r2_all_features - r2_without_feature
    sensitivity_values.append(sensitivity)
    if feature != '':
        sensitivity_rank.append(feature)
        parameters_taken.append(','.join(features_without_feature))
        r2_scores.append(r2_without_feature)
    else:
        sensitivity_rank.append('-')
        parameters_taken.append('All parameters')
        r2_scores.append(r2_all_features)
    
results_df = pd.DataFrame({'Sensitivity Rank': sensitivity_rank,
                           'parameter taken': parameters_taken,
                           'parameter removed':sensitivity_rank,
                           'Sensitivity': sensitivity_values,
                           'R2': r2_scores})

results_df['Sensitivity Rank'] = results_df['Sensitivity'].rank(ascending=False)
new_row_data = {
    'Sensitivity': '-',
    'Rank': '-',
    'parameter taken': 'Passes ,Techniques_Buired, Techniques_Groove,Techniques_Sandwich, Composition_A, Composition_B, Composition_C',
    'Sensitivity Rank': '-',
    'R2': r2_all_features,
    'parameter removed':'-'
}
results_df.loc[-1] = new_row_data
results_df.index = results_df.index + 1  
results_df = results_df.sort_index()  

print(results_df)


c:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the 

  Sensitivity Rank                                    parameter taken  \
0                -  Passes ,Techniques_Buired, Techniques_Groove,T...   
1              1.0  Techniques_Buired,Techniques_Groove,Techniques...   
2              6.0  Passes,Techniques_Groove,Techniques_Sandwich,C...   
3              5.0  Passes,Techniques_Buired,Techniques_Sandwich,C...   
4              4.0  Passes,Techniques_Buired,Techniques_Groove,Com...   
5              2.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   
6              7.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   
7              3.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   

     parameter removed Sensitivity        R2  
0                    -           -  0.744951  
1               Passes    0.675156  0.069796  
2    Techniques_Buired   -0.000368  0.745319  
3    Techniques_Groove   -0.000015  0.744966  
4  Techniques_Sandwich    0.000024  0.744928  
5        Composition_A    0.001157  0.743794  
6        Comp

In [20]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error

features = ['Passes', 'Techniques_Buired', 'Techniques_Groove', 'Techniques_Sandwich', 'Composition_A', 'Composition_B', 'Composition_C']
target = 'R1'
sensitivity_rank = []
mse_scores = []
parameters_taken = []
sensitivity_values = []
cv = KFold(n_splits=5, shuffle=True, random_state=42)
X_all = df[features]
y_all = df[target]
param_grid = {
    'alpha': [0.1, 1.0, 10.0], 
    'fit_intercept': [True, False], 
    'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
    'max_iter': [100, 200, 500], 
    'tol': [1e-3, 1e-4, 1e-5] 
}
grid_search = GridSearchCV(estimator=Ridge(), param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error')
grid_search.fit(X_all, y_all)
mse_all_features = -cross_val_score(grid_search.best_estimator_, X_all, y_all, cv=cv, scoring='neg_mean_squared_error').mean()

for feature in features:
    features_without_feature = [col for col in features if col != feature]
    X = df[features_without_feature]
    y = df[target]
    grid_search = GridSearchCV(estimator=Ridge(), param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error')
    grid_search.fit(X, y)
    
    best_dtr = grid_search.best_estimator_
    mse_without_feature = -cross_val_score(best_dtr, X, y, cv=cv, scoring='neg_mean_squared_error').mean() 
    
    sensitivity = mse_all_features - mse_without_feature
    sensitivity_values.append(sensitivity)
    if feature != '':
        sensitivity_rank.append(feature)
        parameters_taken.append(','.join(features_without_feature))
        mse_scores.append(mse_without_feature)
    else:
        sensitivity_rank.append('-')
        parameters_taken.append('All parameters')
        mse_scores.append(mse_all_features)
    
results_df = pd.DataFrame({'Sensitivity Rank': sensitivity_rank,
                           'parameter taken': parameters_taken,
                           'parameter removed': sensitivity_rank,
                           'Sensitivity': sensitivity_values,
                           'MSE': mse_scores})

results_df['Sensitivity Rank'] = results_df['Sensitivity'].rank(ascending=False)
new_row_data = {
    'Sensitivity': '-',
    'Rank': '-',
    'parameter taken': 'Passes ,Techniques_Buired, Techniques_Groove,Techniques_Sandwich, Composition_A, Composition_B, Composition_C',
    'Sensitivity Rank': '-',
    'MSE': mse_all_features,
    'parameter removed': '-'
}
results_df.loc[-1] = new_row_data
results_df.index = results_df.index + 1  
results_df = results_df.sort_index()  

print(results_df)


c:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the 

  Sensitivity Rank                                    parameter taken  \
0                -  Passes ,Techniques_Buired, Techniques_Groove,T...   
1              7.0  Techniques_Buired,Techniques_Groove,Techniques...   
2              2.0  Passes,Techniques_Groove,Techniques_Sandwich,C...   
3              4.0  Passes,Techniques_Buired,Techniques_Sandwich,C...   
4              5.0  Passes,Techniques_Buired,Techniques_Groove,Com...   
5              3.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   
6              1.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   
7              6.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   

     parameter removed Sensitivity       MSE  
0                    -           -  0.085265  
1               Passes   -0.219177  0.304442  
2    Techniques_Buired    0.000414  0.084851  
3    Techniques_Groove     0.00038  0.084885  
4  Techniques_Sandwich    0.000355  0.084910  
5        Composition_A    0.000386  0.084879  
6        Comp

In [21]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
import numpy as np

features = ['Passes', 'Techniques_Buired', 'Techniques_Groove', 'Techniques_Sandwich', 'Composition_A', 'Composition_B', 'Composition_C']
target = 'R1'
sensitivity_rank = []
rmse_scores = []
parameters_taken = []
sensitivity_values = []
cv = KFold(n_splits=5, shuffle=True, random_state=42)
X_all = df[features]
y_all = df[target]
param_grid = {
    'alpha': [0.1, 1.0, 10.0], 
    'fit_intercept': [True, False], 
    'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'], 
    'tol': [1e-3, 1e-4, 1e-5] 
}
grid_search = GridSearchCV(estimator=Ridge(), param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error')
grid_search.fit(X_all, y_all)
mse_all_features = -cross_val_score(grid_search.best_estimator_, X_all, y_all, cv=cv, scoring='neg_mean_squared_error').mean()

for feature in features:
    features_without_feature = [col for col in features if col != feature]
    X = df[features_without_feature]
    y = df[target]
    grid_search = GridSearchCV(estimator=Ridge(), param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error')
    grid_search.fit(X, y)
    
    best_dtr = grid_search.best_estimator_
    mse_without_feature = -cross_val_score(best_dtr, X, y, cv=cv, scoring='neg_mean_squared_error').mean()
    
    sensitivity = mse_all_features - mse_without_feature
    sensitivity_values.append(sensitivity)
    rmse_without_feature = np.sqrt(mse_without_feature)  
    if feature != '':
        sensitivity_rank.append(feature)
        parameters_taken.append(','.join(features_without_feature))
        rmse_scores.append(rmse_without_feature)
    else:
        sensitivity_rank.append('-')
        parameters_taken.append('All parameters')
        rmse_scores.append(np.sqrt(mse_all_features))
    
results_df = pd.DataFrame({'Sensitivity Rank': sensitivity_rank,
                           'parameter taken': parameters_taken,
                           'parameter removed': sensitivity_rank,
                           'Sensitivity': sensitivity_values,
                           'RMSE': rmse_scores})

results_df['Sensitivity Rank'] = results_df['Sensitivity'].rank(ascending=False)
new_row_data = {
    'Sensitivity': '-',
    'Rank': '-',
    'parameter taken': 'Passes ,Techniques_Buired, Techniques_Groove,Techniques_Sandwich, Composition_A, Composition_B, Composition_C',
    'Sensitivity Rank': '-',
    'RMSE': np.sqrt(mse_all_features),
    'parameter removed': '-'
}
results_df.loc[-1] = new_row_data
results_df.index = results_df.index + 1  
results_df = results_df.sort_index()  

print(results_df)


  Sensitivity Rank                                    parameter taken  \
0                -  Passes ,Techniques_Buired, Techniques_Groove,T...   
1              7.0  Techniques_Buired,Techniques_Groove,Techniques...   
2              4.0  Passes,Techniques_Groove,Techniques_Sandwich,C...   
3              5.0  Passes,Techniques_Buired,Techniques_Sandwich,C...   
4              2.0  Passes,Techniques_Buired,Techniques_Groove,Com...   
5              3.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   
6              1.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   
7              6.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   

     parameter removed Sensitivity      RMSE  
0                    -           -  0.291153  
1               Passes   -0.219672  0.551762  
2    Techniques_Buired   -0.000117  0.291353  
3    Techniques_Groove   -0.000134  0.291383  
4  Techniques_Sandwich   -0.000088  0.291305  
5        Composition_A   -0.000109  0.291341  
6        Comp

In [22]:
import pandas as pd
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
import numpy as np

features = ['Passes', 'Techniques_Buired', 'Techniques_Groove', 'Techniques_Sandwich', 'Composition_A', 'Composition_B', 'Composition_C']
target = 'R1'
sensitivity_rank = []
vaf_scores = []
parameters_taken = []
sensitivity_values = []
cv = KFold(n_splits=5, shuffle=True, random_state=42)
X_all = df[features]
y_all = df[target]
param_grid = {
    'alpha': [0.1, 1.0, 10.0], 
    'fit_intercept': [True, False], 
    'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'], 
    'tol': [1e-3, 1e-4, 1e-5] 
}
grid_search = GridSearchCV(estimator=Ridge(), param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error')
grid_search.fit(X_all, y_all)
y_pred_all_features = cross_val_predict(grid_search.best_estimator_, X_all, y_all, cv=cv)

mse_all_features = mean_squared_error(y_all, y_pred_all_features)
vaf_all_features = (1 - mse_all_features / np.var(y_all))*100

for feature in features:
    features_without_feature = [col for col in features if col != feature]
    X = df[features_without_feature]
    y = df[target]
    grid_search = GridSearchCV(estimator=Ridge(), param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error')
    grid_search.fit(X, y)
    
    best_dtr = grid_search.best_estimator_
    y_pred_without_feature = cross_val_predict(best_dtr, X, y, cv=cv)
    
    mse_without_feature = mean_squared_error(y, y_pred_without_feature)
    vaf_without_feature = (1 - mse_without_feature / np.var(y))*100  
    
    sensitivity = vaf_all_features - vaf_without_feature
    sensitivity_values.append(sensitivity)
    
    if feature != '':
        sensitivity_rank.append(feature)
        parameters_taken.append(','.join(features_without_feature))
        vaf_scores.append(vaf_without_feature)
    else:
        sensitivity_rank.append('-')
        parameters_taken.append('All parameters')
        vaf_scores.append(vaf_all_features)
    
results_df = pd.DataFrame({'Sensitivity Rank': sensitivity_rank,
                           'parameter taken': parameters_taken,
                           'parameter removed': sensitivity_rank,
                           'Sensitivity': sensitivity_values,
                           'VAF': vaf_scores})

results_df['Sensitivity Rank'] = results_df['Sensitivity'].rank(ascending=False)
new_row_data = {
    'Sensitivity': '-',
    'Rank': '-',
    'parameter taken': 'Passes ,Techniques_Buired, Techniques_Groove,Techniques_Sandwich, Composition_A, Composition_B, Composition_C',
    'Sensitivity Rank': '-',
    'VAF': vaf_all_features,
    'parameter removed': '-'
}
results_df.loc[-1] = new_row_data
results_df.index = results_df.index + 1  
results_df = results_df.sort_index()  

print(results_df)


  Sensitivity Rank                                    parameter taken  \
0                -  Passes ,Techniques_Buired, Techniques_Groove,T...   
1              1.0  Techniques_Buired,Techniques_Groove,Techniques...   
2              6.0  Passes,Techniques_Groove,Techniques_Sandwich,C...   
3              4.0  Passes,Techniques_Buired,Techniques_Sandwich,C...   
4              3.0  Passes,Techniques_Buired,Techniques_Groove,Com...   
5              5.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   
6              7.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   
7              2.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   

     parameter removed Sensitivity        VAF  
0                    -           -  75.305101  
1               Passes   64.022981  11.282120  
2    Techniques_Buired   -0.000372  75.305474  
3    Techniques_Groove    0.006024  75.299078  
4  Techniques_Sandwich    0.007432  75.297669  
5        Composition_A    0.004068  75.301033  
6     

In [23]:
import pandas as pd
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np

features = ['Passes', 'Techniques_Buired', 'Techniques_Groove', 'Techniques_Sandwich', 'Composition_A', 'Composition_B', 'Composition_C']
target = 'R1'
sensitivity_rank = []
ios_scores = []
parameters_taken = []
sensitivity_values = []
cv = KFold(n_splits=5, shuffle=True, random_state=42)
X_all = df[features]
y_all = df[target]
param_grid = {
    'alpha': [0.1, 1.0, 10.0], 
    'fit_intercept': [True, False], 
    'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'], 
    'tol': [1e-3, 1e-4, 1e-5] 
}
grid_search = GridSearchCV(estimator=Ridge(), param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error')
grid_search.fit(X_all, y_all)
y_pred_all_features = cross_val_predict(grid_search.best_estimator_, X_all, y_all, cv=cv)

mse_all_features = np.sum((y_all - y_pred_all_features) ** 2) / len(y_all)
ios_all_features = np.sqrt(mse_all_features) / np.mean(y_pred_all_features)

for feature in features:
    features_without_feature = [col for col in features if col != feature]
    X = df[features_without_feature]
    y = df[target]
    grid_search = GridSearchCV(estimator=Ridge(), param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error')
    grid_search.fit(X, y)
    
    best_dtr = grid_search.best_estimator_
    y_pred_without_feature = cross_val_predict(best_dtr, X, y, cv=cv)
    
    mse_without_feature = np.sum((y - y_pred_without_feature) ** 2) / len(y)
    ios_without_feature = np.sqrt(mse_without_feature) / np.mean(y_pred_without_feature)
    
    sensitivity = ios_all_features - ios_without_feature
    sensitivity_values.append(sensitivity)
    
    if feature != '':
        sensitivity_rank.append(feature)
        parameters_taken.append(','.join(features_without_feature))
        ios_scores.append(ios_without_feature)
    else:
        sensitivity_rank.append('-')
        parameters_taken.append('All parameters')
        ios_scores.append(ios_all_features)
    
results_df = pd.DataFrame({'Sensitivity Rank': sensitivity_rank,
                           'parameter taken': parameters_taken,
                           'parameter removed': sensitivity_rank,
                           'Sensitivity': sensitivity_values,
                           'IOS': ios_scores})

results_df['Sensitivity Rank'] = results_df['Sensitivity'].rank(ascending=False)
new_row_data = {
    'Sensitivity': '-',
    'parameter taken': 'Passes ,Techniques_Buired, Techniques_Groove,Techniques_Sandwich, Composition_A, Composition_B, Composition_C',
    'Sensitivity Rank': '-',
    'IOS': ios_all_features,
    'parameter removed': '-'
}
results_df.loc[-1] = new_row_data
results_df.index = results_df.index + 1  
results_df = results_df.sort_index()  

print(results_df)


  Sensitivity Rank                                    parameter taken  \
0                -  Passes ,Techniques_Buired, Techniques_Groove,T...   
1              7.0  Techniques_Buired,Techniques_Groove,Techniques...   
2              3.0  Passes,Techniques_Groove,Techniques_Sandwich,C...   
3              2.0  Passes,Techniques_Buired,Techniques_Sandwich,C...   
4              5.0  Passes,Techniques_Buired,Techniques_Groove,Com...   
5              4.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   
6              1.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   
7              6.0  Passes,Techniques_Buired,Techniques_Groove,Tec...   

     parameter removed Sensitivity       IOS  
0                    -           -  0.200748  
1               Passes    -0.17655  0.377298  
2    Techniques_Buired    0.001983  0.198765  
3    Techniques_Groove    0.001987  0.198761  
4  Techniques_Sandwich    0.001968  0.198780  
5        Composition_A    0.001977  0.198770  
6        Comp